In [8]:
import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')
from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

query = f'''
CREATE TABLE hive.ml_tmp_db.solarland_battle_sample_dd AS (
    battle_id VARCHAR
    , team_id VARCHAR

    , r_elo INTEGER
    , r_lifespan INTEGER
    , r_his_battle_number_3d INTEGER
    
    , feature VARCHAR
    , is_churn INTEGER
    , dt VARCHAR
) 


INSERT INTO hive.ml_tmp_db.solarland_battle_sample_dd

WITH his_stats AS (
    SELECT 
        FORMAT('%d:%d', CAST(GREATEST(elo_start, -1) AS INTEGER) ,  CAST(LEAST(elo_end, 5E3) AS INTEGER)) AS room_elo
        
        , open_id
        , event_time as his_event_time
        , os
        
        , safearea_rank
        , rank_team

        , hit_number
        , hited_number
        , IF( shoot_number > 0, CAST(hit_number AS DOUBLE) / CAST(shoot_number AS DOUBLE), 0.0) as hit_rate
        , headshot_number
        , headshoted_number
        , IF( shoot_number > 0, CAST(headshot_number AS DOUBLE) / CAST(shoot_number AS DOUBLE), 0.0) as headshot_rate
        , damage_amount
        , damage_taken_amount
        , save_number
        
        , weapon1
        , weapon2
        , character_id

        , kill_number
  		, assist_number 
  			
        , dead_number
        , death_number
        
        , IF(survival_time > 0, CAST(kill_number + assist_number AS DOUBLE) / (CAST(survival_time AS DOUBLE) / 60 / 3), -1.0) AS ka_per_3min

        , survival_time

    FROM hive."10060_oss_bi_dw".solarland_battle_end_server
    WHERE dt BETWEEN '{{ ds.add(days = -7).to_date_string() }}' AND '{{ ds.add(days = 0).to_date_string() }}'
        AND mode_id in (5, 7)
        AND warm_game_id = 0
)

, games_data AS (
    SELECT 
        a.battle_id
        , a.team_id
        , a.elo
        , a.event_time
        , d.*
        , COALESCE(DATE_DIFF('day', TRY_CAST(k.role_create_time AS TIMESTAMP), TRY_CAST( a.event_time AS TIMESTAMP)), -1) AS lifespan
        , DATE_DIFF('hour', d.his_event_time, a.event_time) AS fade_time
        , ROW_NUMBER() OVER (PARTITION BY a.battle_id, a.team_id, a.open_id ORDER BY d.his_event_time DESC) AS rnk
        , COUNT() OVER (PARTITION BY a.battle_id, a.team_id, a.open_id) AS his_battle_number
    FROM (
        SELECT *
        FROM hive."10060_oss_bi_dw".solarland_battle_end_server
        WHERE dt = '{{ ds.add(days = 0).to_date_string() }}'
            AND mode_id in (5, 7)
            AND warm_game_id = 0
    ) AS a
    LEFT JOIN his_stats AS d
  	ON d.open_id = a.open_id AND d.his_event_time < a.event_time
    INNER JOIN kudu."10060_bi_ods".total_role_attr_all AS k
    ON a.open_id = k.acc_id
)

, retention AS (
    SELECT open_id, MAX(event_time) AS event_time
    FROM hive."10060_oss_bi_dw".solarland_battle_end_server
    WHERE dt BETWEEN '{{ ds.add(days = 0).to_date_string() }}' AND ''{{ ds.add(days = 1).to_date_string() }}''
        AND mode_id in (5, 7)
    GROUP BY open_id
)

, raw_data AS (
    SELECT 
        battle_id
        , team_id
        , open_id
        , event_time

        , ARBITRARY(elo) AS r_elo
        , ARBITRARY(lifespan) AS r_lifespan
        , ARBITRARY(his_battle_number) AS r_his_battle_number_3d
        
        , CONCAT_WS(
            U&'\0002'
            , FORMAT(U&'fade_time\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', fade_time) ORDER BY rnk DESC)))
            , FORMAT(U&'safearea_rank\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', safearea_rank) ORDER BY rnk DESC)))
            , FORMAT(U&'rank_team\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', rank_team) ORDER BY rnk DESC)))
            
            , FORMAT(U&'hit_rate\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%.2f', hit_rate) ORDER BY rnk DESC)))
            , FORMAT(U&'headshot_rate\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%.2f', headshot_rate) ORDER BY rnk DESC)))

            , FORMAT(U&'hit_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', hit_number) ORDER BY rnk DESC)))
            , FORMAT(U&'hited_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', hited_number) ORDER BY rnk DESC)))
            , FORMAT(U&'headshot_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', headshot_number) ORDER BY rnk DESC)))
            , FORMAT(U&'headshoted_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', headshoted_number) ORDER BY rnk DESC)))
            , FORMAT(U&'damage_amount\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', damage_amount) ORDER BY rnk DESC)))
            , FORMAT(U&'damage_taken_amount\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', damage_taken_amount) ORDER BY rnk DESC)))
            , FORMAT(U&'save_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', save_number) ORDER BY rnk DESC)))

            , FORMAT(U&'weapon1\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', weapon1) ORDER BY rnk DESC)))
            , FORMAT(U&'weapon2\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', weapon2) ORDER BY rnk DESC)))
            , FORMAT(U&'character_id\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', character_id) ORDER BY rnk DESC)))

            , FORMAT(U&'kill_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', kill_number) ORDER BY rnk DESC)))
            , FORMAT(U&'assist_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', assist_number) ORDER BY rnk DESC)))
            , FORMAT(U&'dead_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', dead_number) ORDER BY rnk DESC)))
            , FORMAT(U&'death_number\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', death_number) ORDER BY rnk DESC)))
            
            , FORMAT(U&'ka_per_3min\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%.2f', ka_per_3min) ORDER BY rnk DESC)))
            , FORMAT(U&'survival_time\0003%s', CONCAT_WS(U&'\0004', ARRAY_AGG(FORMAT('%d', survival_time) ORDER BY rnk DESC)))
        ) As feature
    FROM games_data
    WHERE rnk <= 5
    GROUP BY battle_id, team_id, open_id, event_time
)

SELECT 
    d.*
    , IF(r.open_id is null, 1, 0) AS is_churn
    , '{{ ds.to_date_string() }}' AS dt
FROM raw_data AS d
LEFT JOIN retention AS r
ON d.open_id = r.open_id 
    AND d.event_time < r.event_time
'''

client = SQLClient('MIX')
client.execute_trino_query(query)
ds = client.to_pandas()